# Mount Google Drive on Collab

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Download And Install spark dependency

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf "/content/gdrive/My Drive/spark-2.4.5-bin-hadoop2.7.tgz"
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Import dependency

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark # only run after findspark.init()
from pyspark.sql.functions import col, explode
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType
from pyspark.sql.types import StructType
from pyspark.sql.functions import udf
from pyspark.sql import functions
import datetime
from datetime import timedelta
from pyspark.sql.types import ArrayType
from pyspark.sql.functions import split
from pyspark.sql.types import StringType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
spark = SparkSession.builder.getOrCreate()

# Test Code

In [0]:
segdata=spark.read.option("delimiter", "\t").option("header", "true").csv("/content/gdrive/My Drive/Colab Notebooks/data/SegmentAndId.txt")
segdata.show()

+---+-------+
| id|Segment|
+---+-------+
|  1|  a,b,c|
|  2|    b,d|
|  3|  e,a,c|
|  4|  o,d,e|
|  5|      p|
+---+-------+



In [0]:
segdatasatomic=segdata.withColumn("segmentsplit",split("Segment",",")).withColumn("segmentatomic",explode("segmentsplit")).drop("Segment","segmentsplit")
segdatasatomic.show()

+---+-------------+
| id|segmentatomic|
+---+-------------+
|  1|            a|
|  1|            b|
|  1|            c|
|  2|            b|
|  2|            d|
|  3|            e|
|  3|            a|
|  3|            c|
|  4|            o|
|  4|            d|
|  4|            e|
|  5|            p|
+---+-------------+



In [0]:
segdatasatomiccnt=segdatasatomic.withColumn("idsforsed",functions.collect_list("id").over(Window.partitionBy("segmentatomic")))
segdatasatomiccnt.show()

+---+-------------+---------+
| id|segmentatomic|idsforsed|
+---+-------------+---------+
|  3|            e|   [3, 4]|
|  4|            e|   [3, 4]|
|  4|            o|      [4]|
|  5|            p|      [5]|
|  2|            d|   [2, 4]|
|  4|            d|   [2, 4]|
|  1|            c|   [1, 3]|
|  3|            c|   [1, 3]|
|  1|            b|   [1, 2]|
|  2|            b|   [1, 2]|
|  1|            a|   [1, 3]|
|  3|            a|   [1, 3]|
+---+-------------+---------+



In [0]:
segdatasatomiccnt.filter(id>3)